# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import sys
import pandas as pd
from sql_queries import *


In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")

In [5]:
filepath = song_files

In [6]:
df = pd.read_json(filepath[0], typ='series')
df

num_songs                            1
artist_id           ARD7TVE1187B99BFB1
artist_latitude                   None
artist_longitude                  None
artist_location        California - LA
artist_name                     Casual
song_id             SOMZWCG12A8C13C480
title                 I Didn't Mean To
duration                       218.932
year                                 0
dtype: object

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [7]:

song_data = [df['song_id'], df['title'], df['artist_id'] , df['year'], df['duration']]
print(song_data)
try:
    cur.execute(song_table_insert, song_data)
    conn.commit()
except:
    cur.close()
    conn.close()
    raise

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [8]:
artist_data = [df['artist_id'], df['artist_name'], df['artist_location'], df['artist_latitude'], df['artist_longitude']]
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [9]:
try:
    cur.execute(artist_table_insert, artist_data)
    conn.commit()
except:
    e = sys.exc_info()[0]  
    cur.close()
    conn.close()
    raise

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [10]:
log_files = get_files("data/log_data")

In [24]:
filepath = log_files[0]
print(filepath)

/home/workspace/data/log_data/2018/11/2018-11-30-events.json


In [39]:
df =  pd.read_json(filepath, lines=True)
#df=df11.iloc[0]
df11 = df[df['page'] == 'NextSong']
df11


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
5,LMFAO,Logged In,Jacob,M,4,Klein,227.99628,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,I'm In Miami Bitch,200,1543541072796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
6,DJ Dizzy,Logged In,Jacob,M,5,Klein,221.15220,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Sexy Bitch,200,1543541299796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
7,Fish Go Deep & Tracey K,Logged In,Jacob,M,6,Klein,377.41669,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,The Cure & The Cause (Dennis Ferrer Remix),200,1543541520796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
9,M83,Logged In,Jacob,M,7,Klein,96.18240,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Staring At Me,200,1543541897796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
10,Saybia,Logged In,Jacob,M,8,Klein,257.25342,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,The Second You Sleep,200,1543541993796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [40]:
df1 = df['ts']
df1 = pd.to_datetime(df1, unit='ms')
df1 = pd.Series(pd.to_datetime(df1, unit='ms'))


In [41]:
t = [pd.to_datetime(df['ts'], unit='ms'), df1.dt.hour, df1.dt.day, df1.dt.week, df1.dt.month, df1.dt.year, df1.dt.weekday]
#pd.to_datetime(df1, unit='ms').year, pd.to_datetime(df1, unit='ms').weekday]



In [42]:
time_data = (df['ts'], df1.dt.hour, df1.dt.day, df1.dt.week, df1.dt.month, df1.dt.year, df1.dt.weekday)
column_labels = {"start_time":df['ts'], "hour":t[1], "day":t[2], "week":t[3], "month":t[4], "year":t[5], "weekday":t[6]}

In [43]:
time_df = pd.DataFrame.from_dict(column_labels)
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,1543537327796,0,30,48,11,2018,4
1,1543540121796,1,30,48,11,2018,4
2,1543540368796,1,30,48,11,2018,4
3,1543540625796,1,30,48,11,2018,4
4,1543540856796,1,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [44]:
try:
    for i, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))
        conn.commit()
except:
    e = sys.exc_info()[0]
    e
    cur.close()
    conn.close()
    raise        

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [45]:
user_dict = {"user_id":df['userId'], "first_name":df['firstName'], "last_name":df['lastName'], "gender":df['gender'], "level":df['level']}
user_df = pd.DataFrame.from_dict(user_dict)
user_df.head()

,user_id,first_name,last_name,gender,level
0,91,Jayden,Bell,M,free
1,73,Jacob,Klein,M,paid
2,73,Jacob,Klein,M,paid
3,73,Jacob,Klein,M,paid
4,73,Jacob,Klein,M,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [46]:
print(user_table_insert)


try:  
    for i, row in user_df.iterrows():
        if(row['user_id'] != ''):
            cur.execute(user_table_insert, list(row))
except:
    e = sys.exc_info()[0]
    e
    cur.close()
    conn.close()
    raise            
conn.commit()    

insert into users (user_id, first_name, last_name, gender, level) values (%s, %s, %s, %s, %s) ON CONFLICT(user_id) DO UPDATE set first_name=Excluded.first_name, last_name=Excluded.last_name, gender=Excluded.gender


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [47]:
print(song_select)

for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
    print (index, row.song, ",", row.artist, row.length)

for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
#    print (row.song, row.artist)
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    print(songid, artistid)    
    if(songid == "" or songid == None) :
        continue
    # insert songplay record
    #songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location_user_agent
    #artist	auth	firstName	gender	itemInSession	lastName	length	level	location	method	page	registration	sessionId	song	status	ts	userAgent	userId
    songplay_data = (row['ts'], row['userId'], row['level'], songid, artistid, row['sessionId'], row['location'], row['userAgent'])
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

select s1.song_id as songid, a1.artist_id as artistid from songs s1 join artists a1 on s1.artist_id=a1.artist_id where s1.title=%s and a1.name=%s and s1.duration=%s
0 Jim Henson's Dead , Stephen Lynch 182.85669
1 Shell Shock , Manowar 247.562
2 Women Lose Weight (Feat: Slick Rick) , Morcheeba 257.41016
3 Won't Go Home Without You , Maroon 5 231.23546
4 Hey_ Soul Sister , Train 216.76363
5 I'm In Miami Bitch , LMFAO 227.99628
6 Sexy Bitch , DJ Dizzy 221.1522
7 The Cure & The Cause (Dennis Ferrer Remix) , Fish Go Deep & Tracey K 377.41669
8 None , None nan
9 Staring At Me , M83 96.1824
10 The Second You Sleep , Saybia 257.25342
11 Wide Eyes , Local Natives 266.05669
12 La Resistance (Medley) (LP Version) , South Park 112.97914
13 Restless , UNKLE Feat. Josh Homme 307.19955
14 Noc... , Justyna Steczkowska 333.53098
15 Like You , Evanescence 256.91383
16 God Put A Smile Upon Your Face , Coldplay 297.35138
17 Playboy , Hot Chip 333.50485
18 Every Time You Go , Ellie Goulding 205.06077
19 Na

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [48]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.